# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [112]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [113]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethlehem,-1.4558,-48.5044,24.02,94,75,7.72,BR,1659382740
1,1,new norfolk,-42.7826,147.0587,12.73,70,100,2.68,AU,1709782740
2,2,yulara,-25.2406,130.9889,24.89,19,0,3.88,AU,1704382740
3,3,atuona,-9.8000,-139.0333,25.00,81,28,6.68,PF,1635982740
4,4,gualaquiza,-3.4000,-78.5500,27.27,52,33,2.14,EC,1652182740


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [114]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",s="Humidity",c="City",alpha=0.6)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [115]:
# Narrow down cities that fit criteria and drop any results with null values
nice_weather_cities = city_data_df.loc[(city_data_df["Max Temp"] > 15) & (city_data_df["Max Temp"] < 23) & (city_data_df["Wind Speed"] < 6) & (city_data_df["Cloudiness"] < 4)]

# Drop any rows with null values
nice_weather_cities = nice_weather_cities.dropna(how='any')
del nice_weather_cities["City_ID"]

# Display sample data
nice_weather_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,pisco,-13.7000,-76.2167,22.23,68,0,5.66,PE,1652182740
82,norton,-17.8833,30.7000,22.67,43,2,3.35,ZW,1677382740
94,marsaxlokk,35.8419,14.5431,16.71,82,0,2.57,MT,1673782740
124,saint george,37.1041,-113.5841,15.80,45,0,2.06,US,1644982740
143,tura,25.5198,90.2201,15.58,85,0,1.76,IN,1689982740
149,ahipara,-35.1667,173.1667,17.24,67,0,1.11,NZ,1716982740
155,takaka,-40.8500,172.8000,16.42,56,3,1.61,NZ,1716982740
177,cairns,-16.9167,145.7667,22.26,91,2,4.12,AU,1706182740
178,pasighat,28.0667,95.3333,15.75,54,0,1.70,IN,1689982740
184,mareeba,-17.0000,145.4333,19.69,92,2,0.97,AU,1706182740


### Step 3: Create a new DataFrame called `hotel_df`.

In [116]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_weather_cities[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
70,pisco,PE,-13.7000,-76.2167,68,
82,norton,ZW,-17.8833,30.7000,43,
94,marsaxlokk,MT,35.8419,14.5431,82,
124,saint george,US,37.1041,-113.5841,45,
143,tura,IN,25.5198,90.2201,85,
149,ahipara,NZ,-35.1667,173.1667,67,
155,takaka,NZ,-40.8500,172.8000,56,
177,cairns,AU,-16.9167,145.7667,91,
178,pasighat,IN,28.0667,95.3333,54,
184,mareeba,AU,-17.0000,145.4333,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [117]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 3
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search...
pisco - nearest hotel: La Portada
norton - nearest hotel: Village Pakare Paye
marsaxlokk - nearest hotel: Harbour Lodge
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
tura - nearest hotel: No hotel found
ahipara - nearest hotel: YHA Ahipara
takaka - nearest hotel: No hotel found
cairns - nearest hotel: Cairns Queens Court Accommodation
pasighat - nearest hotel: Aane Hotel
mareeba - nearest hotel: The Gateway Hotel
sakakah - nearest hotel: sakakah residence
dhadar - nearest hotel: No hotel found
chicama - nearest hotel: Hotel Wasi
uthal - nearest hotel: No hotel found
najran - nearest hotel: No hotel found
charcas - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
emerald - nearest hotel: Emerald Central Hotel
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
mount isa - nearest hotel: Ibis Styles
gat - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
70,pisco,PE,-13.7000,-76.2167,68,La Portada
82,norton,ZW,-17.8833,30.7000,43,Village Pakare Paye
94,marsaxlokk,MT,35.8419,14.5431,82,Harbour Lodge
124,saint george,US,37.1041,-113.5841,45,"The Advenire, an Autograph Collection Hotel"
143,tura,IN,25.5198,90.2201,85,No hotel found
149,ahipara,NZ,-35.1667,173.1667,67,YHA Ahipara
155,takaka,NZ,-40.8500,172.8000,56,No hotel found
177,cairns,AU,-16.9167,145.7667,91,Cairns Queens Court Accommodation
178,pasighat,IN,28.0667,95.3333,54,Aane Hotel
184,mareeba,AU,-17.0000,145.4333,92,The Gateway Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [118]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",s="Humidity",c="City",alpha=0.6,hover_cols=["Hotel Name","Country"])

# Display the map
map_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)